In [1]:
from google.colab import drive
drive.mount('/content/drive')
!unrar x '/content/drive/My Drive/colab/price_max_1d.rar'

Mounted at /content/drive

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/My Drive/colab/price_max_1d.rar

Creating    price_max_1d                                              OK
Creating    price_max_1d/data                                         OK
Extracting  price_max_1d/data/A.csv                                        0%  OK 
Extracting  price_max_1d/data/AAL.csv                                      0%  OK 
Extracting  price_max_1d/data/AAPL.csv                                     0%  OK 
Extracting  price_max_1d/data/ABBV.csv                                     0%  OK 
Extracting  price_max_1d/data/ABNB.csv                                     0%  OK 
Extracting  price_max_1d/data/ABT.csv                                      0%  OK 
Extracting  price_max_1d/data/ACGL.csv                                     1%  OK 
Extracting  price_max_1d/data/ACN.csv       

In [1]:
from tensorflow.keras.utils import Sequence
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from tensorflow.keras.layers import Concatenate,Embedding ,Dense ,Input,LSTM,Permute,Softmax,Lambda,Flatten,GRU,Dropout,BatchNormalization, Normalization, Attention, Bidirectional, Masking, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
!pip install keras_self_attention
from keras_self_attention import SeqSelfAttention
from tensorflow.keras import Model
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
print("Num GPUs Available: ", tf.config.list_physical_devices('TPU'))


Num GPUs Available:  []


In [2]:
root_path = '../../..'
period = 'max'
interval = '1d'
prefix = '/kaggle/input/price-1d-transformer'

class DataGenerator(Sequence):

    def __init__(self, meta_df, data_dict, batch_size=64, time_series=30, input_shape=(20,13), output_shape=(20,4), partition_factor=1, **kwargs):
        super().__init__(**kwargs)
        self.ds_pointer = 0
        self.batch_size = batch_size
        self.meta_df = meta_df
        self.time_series = time_series
        self.partition_factor = partition_factor
        self.data = data_dict
        self.load_data() # Load data only when empty
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.input_buffer = np.zeros((batch_size,input_shape[0],input_shape[1]))
        self.label_buffer = np.zeros((batch_size,output_shape[0],output_shape[1]))
        self.reset_pointer()

    def load_data(self):
        if len(self.data) == 0:
          #self.meta_df['Label'] = self.meta_df['Label'].astype(int)
          symbol = self.meta_df['Symbol'].unique()
          for s in tqdm(symbol):
            self.data[s] = pd.read_csv(f'{prefix}/price_{period}_{interval}_transformer/data/{s}.csv')

    def reset_pointer(self, seed = 1314):
        self.meta_df = self.meta_df.sample(frac=1, random_state=seed)
        self.ds_pointer = 0

    def __len__(self):
        return len(self.meta_df)// self.batch_size// self.partition_factor

    def __getitem__(self, index):
        if self.ds_pointer + self.batch_size >= len(self.meta_df):
            self.reset_pointer()
            return self.__getitem__(index)

        to_get = self.meta_df.iloc[self.ds_pointer:self.ds_pointer+self.batch_size]
        to_get_symbol = to_get['Symbol'].values
        to_get_index = to_get['Index'].values

        for i, (idx, symbol) in enumerate(zip(to_get_index, to_get_symbol)):
            # Fit transform for OHLC local
            base_data = self.data[symbol].iloc[idx-self.input_shape[0]:idx+self.output_shape[0]].values
            scaler = StandardScaler().fit(base_data[0:self.input_shape[0] ,0:5]) # Label values not involved
            base_data[:, 0:5] = scaler.transform(base_data[:, 0:5])

            self.input_buffer[i, :, :] = base_data[0:self.input_shape[0], :]
            self.label_buffer[i, :, :] = base_data[self.input_shape[0]:, 0:4] # High and Low only

        batch_x = self.input_buffer
        batch_y = self.label_buffer

        self.ds_pointer += self.batch_size

        return batch_x, batch_y

In [3]:
batch_size = 16

print(os.getcwd())
input_shape = (100,24)
output_shape = (20,4)
meta_path = f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/meta.csv'
meta_df = pd.read_csv(meta_path)

# train_meta, test_meta = train_test_split(meta_df, test_size=0.2)
# train_meta.to_csv(f'{root_path}/output/analysis/price_{period}_{interval}/train_meta.csv',index=False)
# test_meta.to_csv(f'{root_path}/output/analysis/price_{period}_{interval}/test_meta.csv',index=False)

train_meta = pd.read_csv(f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/train_meta.csv')
test_meta = pd.read_csv(f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/test_meta.csv')

c:\Users\homan\coding_projects\python_projects\stock_signal_system\script\analysis\prediction


In [33]:
def create_tfrecord(meta_df, output_path):
    with tf.io.TFRecordWriter(output_path) as writer:
        for symbol in tqdm(meta_df['Symbol'].unique()):
            data = pd.read_csv(f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/data/{symbol}.csv', encoding='utf-8')
            start_indexies = meta_df[meta_df['Symbol']==symbol]['Index']
            for idx in start_indexies:
                base_data = data.iloc[idx:idx+input_shape[0]].values
                target_data = data.iloc[idx:idx+input_shape[0]+output_shape[0]].values
                scaler = StandardScaler().fit(base_data[: ,0:5]) # Label values not involved
                target_data[:, 0:5] = scaler.transform(target_data[:, 0:5])
                
                input_data = target_data[0:input_shape[0], :]
                label_data = target_data[input_shape[0]:, 0:4]

                feature = {
                    'input': tf.train.Feature(float_list=tf.train.FloatList(value=input_data.flatten())),
                    'label': tf.train.Feature(float_list=tf.train.FloatList(value=label_data.flatten()))
                }
                example = tf.train.Example(features=tf.train.Features(feature=feature))
                writer.write(example.SerializeToString())
                                # Debugging: Print the first few examples

#create_tfrecord(test_meta, f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/test.tfrecord')
create_tfrecord(train_meta, f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/train.tfrecord')


  0%|          | 0/500 [00:00<?, ?it/s]

In [44]:
def parse_tfrecord_function(example_proto, input_shape, output_shape):
    feature_description = {
        'input': tf.io.FixedLenFeature([input_shape[0] * input_shape[1]], tf.float32),
        'label': tf.io.FixedLenFeature([output_shape[0] * output_shape[1]], tf.float32)
    }
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    
    input_data = tf.reshape(parsed_features['input'], input_shape)
    label_data = tf.reshape(parsed_features['label'], output_shape)
    print(input_data.shape)
    return input_data, label_data

def create_dataset(tfrecord_path, input_shape, output_shape):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path, num_parallel_reads=tf.data.experimental.AUTOTUNE)
    parsed_dataset = raw_dataset.map(lambda x: parse_tfrecord_function(x, input_shape, output_shape), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    parsed_dataset = parsed_dataset.cache()
    dataset = parsed_dataset.repeat().batch(32, num_parallel_calls=tf.data.experimental.AUTOTUNE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

train_data = create_dataset(f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/train.tfrecord', input_shape=input_shape, output_shape=output_shape)
test_data  = create_dataset(f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/test.tfrecord' , input_shape=input_shape, output_shape=output_shape)

for batch in train_data.take(1):
    inputs, labels = batch
    print("Input shape:", inputs.shape)
    print("Label shape:", labels.shape)
    print("Sample input data:", inputs.numpy()[0])
    print("Sample label data:", labels.numpy()[0])

(100, 24)
(100, 24)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: input.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 

In [42]:
def inspect_tfrecord(tfrecord_path, num_records=1):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
    for raw_record in raw_dataset.take(num_records):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        print(example)

# Inspect the TFRecord file
inspect_tfrecord(f'{root_path}/ouput/analysis/price_{period}_{interval}_transformer/train.tfrecord', num_records=5)

features {
  feature {
    key: "label"
    value {
      float_list {
        value: -0.219791725
        value: -0.261346102
        value: -0.126207873
        value: -0.240244791
        value: -0.103233315
        value: -0.245842263
        value: -0.206266522
        value: -0.233891472
        value: -0.244993597
        value: 0.0952479839
        value: -0.142546073
        value: 0.194946513
        value: 0.504762411
        value: 0.631689548
        value: 0.623726726
        value: 0.731789529
        value: 1.45613396
        value: 1.75573659
        value: 1.43084562
        value: 1.93253684
        value: 2.00112486
        value: 2.29062748
        value: 2.12359524
        value: 2.3518455
        value: 2.23109198
        value: 2.12008262
        value: 2.2330637
        value: 2.24066567
        value: 2.23109198
        value: 2.16659522
        value: 2.29351521
        value: 2.32325625
        value: 2.3382
        value: 2.47047544
        value: 2.4928443

In [5]:
train_data = DataGenerator(train_meta, batch_size=batch_size, input_shape=input_shape, partition_factor=1)
test_data = DataGenerator(test_meta, batch_size=batch_size, input_shape=input_shape, partition_factor=100)

  0%|          | 0/502 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

In [ ]:
inp, oup = test_data.__getitem__(0)
print(inp[0][-1])
print(oup[0][:, :])

[1.59789568 1.61801269 1.61569245 1.64724153 0.68811747 0.
 0.         0.         0.         0.         1.82907495 1.1973287
 0.28199742 0.26945921 1.05274203 1.         0.         1.
 0.        ]
[[1.64053256 1.62428734 1.64249015 1.61750086]
 [1.58564654 1.56154181 1.55587694 1.5422045 ]
 [1.55196108 1.53597858 1.55444123 1.55282608]
 [1.52699101 1.56061135 1.52357575 1.56793186]
 [1.58965053 1.56525936 1.58769821 1.57383299]
 [1.57740163 1.57502023 1.59009125 1.58492723]
 [1.58352642 1.57804138 1.60612188 1.58445527]
 [1.57245474 1.55108369 1.56353284 1.54598068]
 [1.5529028  1.55178072 1.5774099  1.55424177]
 [1.5517258  1.54736599 1.57142906 1.55353445]
 [1.54465833 1.5292389  1.55228745 1.55329786]
 [1.5696281  1.54318258 1.57095009 1.55943488]
 [1.56844993 1.57990003 1.59415832 1.59153603]
 [1.59601097 1.59035787 1.61736702 1.59342456]
 [1.59860239 1.58175949 1.60683957 1.60333829]
 [1.62333612 1.62893471 1.63818278 1.62009671]
 [1.61250047 1.59686497 1.62741616 1.61230776]
 [1.

In [9]:
@register_keras_serializable()
def r2_score(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

@register_keras_serializable()
def mda(y_true, y_pred):
    # Extract Open and Close values
    open_true = y_true[:, :, 0]
    close_true = y_true[:, :, 3]
    open_pred = y_pred[:, :, 0]
    close_pred = y_pred[:, :, 3]

    # Determine the direction (1 if Close > Open, 0 otherwise)
    direction_true = K.cast(close_true > open_true, 'float32')
    direction_pred = K.cast(close_pred > open_pred, 'float32')

    # Compare the directions
    correct_directions = K.equal(direction_true, direction_pred)

    # Calculate the accuracy
    accuracy = K.mean(K.cast(correct_directions, 'float32'))

    return accuracy

@register_keras_serializable()
def custom_combined_loss(y_true, y_pred, alpha=0.2):
    # Convert to float32
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    # Calculate MAE
    abs_diff = K.abs(y_true - y_pred)
    mae_value = K.mean(abs_diff)
    mae_value = mae_value / (K.max(abs_diff) + K.epsilon())

    # Calculate Direction Accuracy loss
    direction_accuracy = mda(y_true, y_pred)
    direction_loss = 1 - direction_accuracy
    direction_loss = direction_loss / (K.max(direction_loss) + K.epsilon())

    # Combine the losses
    combined_loss = (1-alpha) * mae_value + alpha * direction_loss

    return combined_loss

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Add()([x, inputs])

    # Feed Forward Part
    y = LayerNormalization(epsilon=1e-6)(x)
    y = Dense(ff_dim, activation="relu", kernel_initializer=HeNormal())(y)
    y = Dropout(dropout)(y)
    y = Dense(inputs.shape[-1], kernel_initializer=HeNormal())(y)
    return Add()([y, x])

def build_model(input_shape, head_size, num_of_labels, num_heads, ff_dim, num_layers, dropout=0):
    #K.clear_session()
    inputs = Input(shape=input_shape)
    mask = Masking(mask_value=0, input_shape=(time_series,num_of_labels),name='Mask_Zeroes')(inputs)
    norm = Normalization(mean=mean, variance=variance,name='Normalization')
    norm = norm(mask)
    x = norm
    for _ in range(num_layers):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    
    x = GlobalAveragePooling1D(keepdims=False)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    # Classification
    outputs = Dense(num_of_labels, activation='softmax', kernel_initializer=HeNormal())(x)

    model = Model(inputs = inputs , outputs = outputs)
    return model

input_shape = input_shape
head_size = 64
num_heads = 20
ff_dim = 256
num_layers = 2
dropout = 0.20
model = build_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout)
#model.load_weights(f'{neural_path}/{save_path_dir}/saved_model.keras')

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input (InputLayer)          [(None, 20, 19)]             0         []                            
                                                                                                  
 dense (Dense)               (None, 20, 256)              5120      ['Input[0][0]']               
                                                                                                  
 bidirectional (Bidirection  [(None, 20, 512),            789504    ['dense[0][0]']               
 al)                          (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                              

In [10]:
# Directory Config
neural_path = 'neural_network'
log_path = 'neural_network/log'
save_path_dir='attention_t20_encode_decode_4regression'

if not os.path.exists(log_path):
    os.makedirs(log_path)
checkpoint = ModelCheckpoint(
    f'{neural_path}/{save_path_dir}/saved_model.keras',
    verbose=1,
    monitor='val_mda',
    save_weights_only=False,
    save_best_only=True,
    initial_value_threshold=0.9850,
    mode='max')

checkpoint2 = ModelCheckpoint(
    f'/content/drive/My Drive/colab/{neural_path}/{save_path_dir}/saved_model.keras',
    verbose=1,
    monitor='val_mda',
    save_weights_only=False,
    save_best_only=True,
    initial_value_threshold=0.9850,
    mode='max')

earlystop = EarlyStopping(
    monitor="val_r2_score",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=False
)
!mkdir neural_network
!mkdir "neural_network/attention_t20_encode_decode_4regression"
tensorboard = TensorBoard(log_dir=log_path)

mkdir: cannot create directory ‘neural_network’: File exists
mkdir: cannot create directory ‘neural_network/attention_t20_encode_decode_4regression’: File exists


In [12]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
# For better performance
def generator_wrapper(generator):
    for data, labels in generator:
        yield data, labels
output_shape=(20,4)
# Convert to tf.data.Dataset and apply prefetching for training data
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: generator_wrapper(train_data),
    output_signature=(
        tf.TensorSpec(shape=(None, *input_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(None, *output_shape), dtype=tf.float32)
    )
).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Convert to tf.data.Dataset and apply prefetching for validation data
val_dataset = tf.data.Dataset.from_generator(
    generator=lambda: generator_wrapper(test_data),
    output_signature=(
        tf.TensorSpec(shape=(None, *input_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(None, *output_shape), dtype=tf.float32)
    )
).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [ ]:
history = model.fit(train_data, batch_size=batch_size, epochs=2000, validation_data=test_data, callbacks=[checkpoint, checkpoint2, earlystop, tensorboard])
plt.plot(history.history['r2_score'])
plt.plot(history.history['val_r2_score'])
plt.plot(history.history['mda'])
plt.plot(history.history['val_mda'])

plt.title('model R2')
plt.ylabel('R2')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print(f"Highest Val R2: {max(history.history['val_r2_score'])}")
print(f"Highest Val MDA: {max(history.history['val_mda'])}")

Epoch 1/2000
 2006/13114 [===>..........................] - ETA: 51:50 - loss: 0.0374 - r2_score: 0.9945 - mda: 0.9842

In [ ]:
from tensorflow.keras.models import load_model
model = load_model(f'{neural_path}/{save_path_dir}/saved_model_epoch4.keras')

In [ ]:
test_x, test_y = test_data.__getitem__(0)
predict_result = model.predict(test_x)
print(f"Actual: Max: {max(test_y[0][:,1])}, Min: {min(test_y[0][:,2])}, ArgMax: {np.argmax(test_y[0][:,-1])}, ArgMin: {np.argmin(test_y[0][:,-1])}")
print(f"Predict: Max: {max(predict_result[0][:,1])}, Min:{min(predict_result[0][:,2])}, ArgMax: {np.argmax(predict_result[0][:,-1])}, ArgMin: {np.argmin(predict_result[0][:,-1])}")
print(f"Model feed: Max: {max(test_x[0][:,1])}, Min: {min(test_x[0][:,2])}")
print(test_y[0][:,2].shape)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Actual: Max: 0.9679888273830812, Min: 0.9062574765036792, ArgMax: 10, ArgMin: 17
Predict: Max: 0.522071897983551, Min:0.4985039234161377, ArgMax: 19, ArgMin: 0
Model feed: Max: 0.925346474934392, Min: 0.8341400514156476
(20,)


In [ ]:
#Transfer Learning
symbol = "MMM"
meta_path = f'{root_path}/ouput/analysis/price_{period}_{interval}/meta.csv'
meta_df = pd.read_csv(meta_path)
meta_df = meta_df[meta_df['Symbol']==symbol]

train_meta, test_meta = train_test_split(meta_df, test_size=0.2)
train_data = DataGenerator(train_meta, batch_size=batch_size, input_shape=input_shape, partition_factor=1)
test_data = DataGenerator(test_meta, batch_size=batch_size, input_shape=input_shape, partition_factor=1)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Directory Config
neural_path = 'neural_network'
log_path = 'neural_network/log'
save_path_dir='neural_network_transfer'

if not os.path.exists(log_path):
    os.makedirs(log_path)
checkpoint = ModelCheckpoint(
    f'{neural_path}/{save_path_dir}/saved_model_{symbol}.keras',
    verbose=1,
    monitor='val_r2_score',
    save_weights_only=False,
    save_best_only=True,
    mode='max')

earlystop = EarlyStopping(
    monitor="val_r2_score",
    min_delta=0,
    patience=20,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=False
)
!mkdir neural_network

tensorboard = TensorBoard(log_dir=log_path)


history = model.fit(train_data, batch_size=batch_size, epochs=2000, validation_data=test_data, callbacks=[checkpoint, earlystop, tensorboard])
plt.plot(history.history['r2_score'])
plt.plot(history.history['val_r2_score'])
plt.title('model R2')
plt.ylabel('R2')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print(f"Highest Val R2: {max(history.history['val_r2_score'])}")

�l�ؿ����ɮ� neural_network �w�g�s�b�C


Epoch 1/2000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - loss: 0.0026 - r2_score: 0.9976
Epoch 1: val_r2_score improved from -inf to 0.99764, saving model to neural_network/neural_network_transfer/saved_model_MMM.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 18s 350ms/step - loss: 0.0026 - r2_score: 0.9976 - val_loss: 0.0024 - val_r2_score: 0.9976
Epoch 2/2000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step - loss: 0.0023 - r2_score: 0.9978
Epoch 2: val_r2_score did not improve from 0.99764
48/48 ━━━━━━━━━━━━━━━━━━━━ 22s 447ms/step - loss: 0.0023 - r2_score: 0.9978 - val_loss: 0.0028 - val_r2_score: 0.9973
Epoch 3/2000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step - loss: 0.0022 - r2_score: 0.9979
Epoch 3: val_r2_score improved from 0.99764 to 0.99774, saving model to neural_network/neural_network_transfer/saved_model_MMM.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 30s 601ms/step - loss: 0.0022 - r2_score: 0.9979 - val_loss: 0.0024 - val_r2_score: 0.9977
Epoch 4/2000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step - loss: 0.0024

KeyboardInterrupt: 